# Adding multiple sources to the problem setup

This notebook uses the water tank model from the previous notebook and showcase a possible solution to generating source positions and mesurement setups.

In [ ]:

import numpy as np
import numpy.typing as npt
import math
import matplotlib.pyplot as plt 
from examples.seismic import Receiver
from examples.seismic import RickerSource
from examples.seismic import Model, plot_velocity, TimeAxis
from devito import TimeFunction
from devito import Eq, solve
from devito import Operator
from devito import ConditionalDimension


# Grid setup

Set velocity model and global parameters for the water tank.

In [ ]:

nx = 101
nz = 101
nb = 10
shape = (nx, nz)
spacing = (.003, .003) #30x30 cm
origin = (0., 0.)
glass = 5
# Define a velocity profile. The velocity is in km/s
v = np.empty(shape, dtype=np.float32)
v[:, :] = 1.5 # v in distilled water 1496.7 m/s
v[:, -glass:] = 5.64 # v in glass 5640 m/s
v[:glass, :] = 5.64 # v in glass 5640 m/s
v[-glass:, :] = 5.64 # v in glass 5640 m/s
# Define an object's velocity profile
r = 15
a, b = shape[0] / 2, shape[1] - r - glass
y, x = np.ogrid[-a:shape[0]-a, -b:shape[1]-b]
v[x*x + y*y <= r*r] = 3.24

model = Model(vp=v, origin=origin, shape=shape, spacing=spacing,
              space_order=2, nbl=10, bcs="damp")

# Define the positions of the sources and receivers

In [ ]:

def srcPositions(cx: float, cy:float, alpha:float, ns:int, sdist:float) -> np.typing.NDArray:
    assert alpha >= 0 and alpha < 180
    assert ns > 0
    dx = sdist * math.sin(math.pi / 180 * alpha)
    dy = sdist * math.cos(math.pi / 180 * alpha)

    res = np.zeros((ns, 2))
    res[:, 0] = np.linspace(cx - dx * (ns - 1) / 2, cx + dx * (ns - 1) / 2, num = ns)
    res[:, 1] = np.linspace(cy - dy * (ns - 1) / 2, cy + dy * (ns - 1) / 2, num = ns)
    return res

In [ ]:

# Set time range, source, source coordinates and receiver coordinates
t0 = 0.  # Simulation starts a t=0
tn = 2000.  # Simulation lasts tn milliseconds
dt = model.critical_dt  # Time step from model grid spacing
time_range = TimeAxis(start=t0, stop=tn, step=dt)
nt = time_range.num  # number of time steps

f0 = 0.010  # Source peak frequency is 10Hz (0.010 kHz)

ns = 5 # number of sources
nr = 101 # number of receivers
depth = .0001
source_distance = .01 # spacing of the sources in meters
alpha = 90 # angle of the sources to the water surface (0° - 180°) 90° means sources are parallel with the water surface
cx = model.domain_size[0] * 0.5
cy = (ns - 1) / 2 * source_distance
pos = srcPositions(cx, cy, alpha, ns, source_distance)
src = RickerSource(
    name='src',
    grid=model.grid,
    f0=f0,
    time_range=time_range,
    npoint=ns)  


src.coordinates.data[:] = pos[:]

rec = Receiver(
    name='rec',
    grid=model.grid,
    npoint=nr,
    time_range=time_range)  # new
rec.coordinates.data[:, 0] = np.linspace(0, model.domain_size[0], 
                                         num=nr)
rec.coordinates.data[:, 1] = depth


plot_velocity(model, source=src.coordinates.data,
              receiver=rec.coordinates.data[::4, :])
